In [177]:
from astropy.coordinates import Angle
from astropy.table import QTable, vstack
import astropy.units as u

In [135]:
# read in the text format version of the WDS and save as an Astropy QTable
with open("wdsweb_summ2.txt", "r", newline="\n") as f:
    for _ in range(5):
        f.readline()
    rows=[]
    for line in f.readlines():
        try:
            row = {}
            row["ID"] = line[0:10]
            row["Separation"] = (float(line[46:51]) + float(line[51:57])) / 2 * u.arcsec
            row["Mag A"] = float(line[58:63])
            row["Mag B"] = float(line[63:69])
            row["Mag Avg"] = (row["Mag A"] + row["Mag B"]) / 2
            row["Spectral Type"] = line[70:79]
            row["RA"] = Angle(f"{line[112:114]}h{line[114:116]}m{line[116:121]}s")
            row["Dec"] = Angle(f"{line[121:124]}d{line[124:126]}m{line[126:130]}s")
            rows.append(row)
        except:
            continue
wds = QTable(rows)
wds["Separation"].info.format = ".0f"
wds["Mag A"].info.format = ".2f"
wds["Mag B"].info.format = ".2f"
wds["Mag Avg"].info.format = ".2f"
wds["RA"].info.format = "12.8f"
wds["Dec"].info.format = "13.8f"
wds.write("wdsweb_summ2.ecsv", overwrite=True)
wds

In [209]:
# apply target selection criteria
criteria = (wds["Separation"] > 45 * u.arcsec) & (wds["Separation"] < 4 * u.arcmin) & (wds["Mag Avg"] < 9)
filtered = wds[criteria]

# cluster the list by separation in groups of 10 arcsec, and take the first n brightest targets from each cluster
n = 10
filtered["Sep Category"] = (filtered["Separation"].value / 10).astype(int) * 10
filtered.sort(["Mag Avg"])
target_list = vstack([group[0:n] for group in filtered.group_by("Sep Category").groups])

target_list

ID,Separation,Mag A,Mag B,Mag Avg,Spectral Type,RA,Dec,Sep Category
,arcsec,,,,,hourangle,deg,
str10,float64,float64,float64,float64,str9,float64,float64,int32
18501+3322,47,3.63,6.69,5.16,B7Ve+A8p,18.83466389,33.36266667,40
05354-0555,49,2.77,9.81,6.29,+A0V,5.59055000,-5.90988889,40
01144-0755,49,5.19,7.85,6.52,F6V+G9V,1.24001111,-7.92283333,40
18138-2104,45,3.85,9.22,6.54,B8Iape,18.22939167,-21.05883333,40
10314-5343,48,4.95,8.58,6.77,F6V,10.52272778,-53.71547222,40
08025+6305,49,6.15,7.53,6.84,G1III,8.04189444,63.09033333,40
05597+3713,46,2.60,11.10,6.85,A0pSi,5.99534444,37.21275000,40
18549+3358,46,6.14,7.60,6.87,A8,18.91458889,33.96858333,40


In [210]:
# filter the targets for observability

import astroplan as ap
from astropy.coordinates import SkyCoord
from astropy.time import Time


def calc_observability(
    targets: SkyCoord, observer: ap.Observer, observing_window: list[tuple[Time, Time]], constraints: list[ap.Constraint] = None
) -> list[bool]:

    # validate inputs
    if isinstance(targets, SkyCoord):
        target_coords = targets
    else:
        raise ValueError()
    if constraints is None:
        constraints = [
            ap.AltitudeConstraint(30 * u.deg, 80 * u.deg),
            ap.AirmassConstraint(2),
            # ap.AtNightConstraint.twilight_civil(),
        ]

    for beg, end in observing_window:
        observability = ap.is_observable(constraints, observer, targets, (beg, end))
        return observability


observer = ap.Observer.at_site("Apache Point Observatory", timezone="US/Mountain")

session_beg, session_end = observer.tonight(Time("2024-09-16", format="iso", scale="utc"), horizon=-6 * u.deg)
session_end -= (session_end - session_beg) / 2

target_list["Observable"] = calc_observability(
    SkyCoord(target_list["RA"], target_list["Dec"], unit="deg"), observer, [(session_beg, session_end)]
)
target_list[target_list["Observable"]]

ID,Separation,Mag A,Mag B,Mag Avg,Spectral Type,RA,Dec,Sep Category,Observable
,arcsec,,,,,hourangle,deg,,
str10,float64,float64,float64,float64,str9,float64,float64,int32,bool
18501+3322,47,3.63,6.69,5.16,B7Ve+A8p,18.83466389,33.36266667,40,True
01144-0755,49,5.19,7.85,6.52,F6V+G9V,1.24001111,-7.92283333,40,True
18138-2104,45,3.85,9.22,6.54,B8Iape,18.22939167,-21.05883333,40,True
18549+3358,46,6.14,7.60,6.87,A8,18.91458889,33.96858333,40,True
17208-1251,48,4.33,9.44,6.88,A2V,17.34712778,-12.84688889,40,True
18138-2104,47,3.85,9.96,6.91,B8Iape,18.22939167,-21.05883333,40,True
18006+0256,55,3.96,8.06,6.01,B5I+B2V,18.01075556,2.93158333,50,True
18369+3846,61,0.09,9.50,4.79,A0Va,18.61564722,38.78366667,60,True
